In [1]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2336803 sha256=c0b7fefe5fac451433e82a979786c02744f68404be143df05dc14a400daf20ef
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [2]:
%%time

model_id = "TheBloke/Llama-2-13B-chat-GGML"

from ctransformers import AutoModelForCausalLM

# check ctransformers doc for more configs
config = {'max_new_tokens': 256, 'repetition_penalty': 1.1,
          'temperature': 0.5, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(
      model_id,
      model_type="llama",
      # lib='avx2', # for cpu use
      gpu_layers=130, #110 for 7b, 130 for 13b
      **config
      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1.83 s, sys: 1.05 s, total: 2.88 s
Wall time: 3.24 s


In [5]:
%%time

prompt = """I want to get a score of semantic similarity of two sentences. Example [Anna is hungry, Anna wants some food] = 1, [Doctors work hard, I ride a car] = 0.
            What is similarity score of [Trees are gree, Forrests consist of trees]?"""

# prompt = "Assess semantical similarity of two sentences: [Anna is hungry, Anna wants some food]. Answer with a single number from 0 to 1"

tokens = llm.tokenize(prompt)
llm(prompt, stream=False)

CPU times: user 30.9 s, sys: 1.59 s, total: 32.5 s
Wall time: 19.8 s


'\n\\end{code}\n\nYou can use the following code to calculate the semantic similarity between two sentences:\n```\nfrom nltk.similarity import cosine_similarity\n\ndef calculate_semantic_similarity(sentence1, sentence2):\n    # Tokenize the sentences\n    tokens1 = nltk.word_tokenize(sentence1)\n    tokens2 = nltk.word_tokenize(sentence2)\n\n    # Create a list of tuples containing the tokens and their frequencies\n    token_freqs1 = [(t, len(tokens1)) for t in tokens1]\n    token_freqs2 = [(t, len(tokens2)) for t in tokens2]\n\n    # Calculate the similarity using cosine similarity\n    sim = cosine_similarity(token_freqs1, token_freqs2)\n\n    return sim\n\n# Example usage\nsentence1 = "Anna is hungry, Anna wants some food"\nsentence2 = "Doctors work hard, I ride a car"\n\nsem_sim = calculate_semantic_'

# Unstable results - another take with microsoft guidance library

In [1]:
import requests

def download_file_with_progress(url, filename):
    """
    Download a file with progress indicator from a given URL

    :param url: URL to the file
    :param filename: Filename to save the downloaded content
    """
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024*1024*100 # 100 megabites chunks
    progress_bar_size = 50
    print(f"Starting download of {filename}")

    with open(filename, 'wb') as file:
        downloaded_size = 0
        for data in response.iter_content(block_size):
            downloaded_size += len(data)
            file.write(data)
            done = int(progress_bar_size * downloaded_size / total_size_in_bytes)
            print(f"\r[{'█' * done}{'.' * (progress_bar_size - done)}] {downloaded_size * 100 / total_size_in_bytes:.2f}%", end = '')
    print("\nDownload completed.")

# URL to the .gguf file
gguf_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q3_K_L.gguf?download=true"

# Local filename to save the .gguf file
gguf_filename = "llama-2-7b-chat.Q3_K_L.gguf"

# Download the .gguf file with progress
download_file_with_progress(gguf_url, gguf_filename)


Starting download of llama-2-7b-chat.Q3_K_L.gguf
[██████████████████████████████████████████████████] 100.00%
Download completed.


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install guidance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.26-cp310-cp310-manylinux_2_35_x86_64.whl size=8129950 sha256=7b57e3b3c75a77fe51a9b3e40d59ead5019fbc880c63a4f786fb85bb37b4a616
  Stored in directory: /root/.cache/pip/wheels/91/80/ce/ac6afea8c1d6fbcec7e14183033a5b2796c742d4f470010c72
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 12.5 MB/s eta 0:00:0

In [10]:
import guidance
from guidance import models, gen, system, user, assistant

# A sample model can be downloaded from
# https://huggingface.co/TheBloke/Llama-2-7B-GGUF/blob/main/llama-2-7b.Q5_K_M.gguf
llama2 = models.LlamaCppChat(gguf_filename, n_gpu_layers=-1, n_ctx=4096)

# llama2 = models.Transformers("TheBloke/Llama-2-7B-GGML")
# gpt3 = models.OpenAI("text-davinci-003")
#palm2 = models.VertexAI("text-bison@001")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [11]:
query = """How semantically similar are those two sentences on scale [0-1]:
        Dogs eat bones; Doctors work hard"""
llama2 + f'''\
Q: {query}
A: {gen(stop="Q:")}'''

In [21]:
@guidance
def simil(lm, sentences):
    with system():
        lm += """We want to assess the semantic similarity of sentence pairs.
        For example the similarity of a pair [Anna is hungry, Anna wants to eat a dinner] = 0.95,
        the similarity of a pair [Dogs like balls, Doctors work hard] = 0"""

    with user():
        lm += f"""\
        Assess the similarity of the pair:
        {sentences}
        Similarity score of this pair in range from 0 to 1 is equall to:
        """

    with assistant():
        lm += gen(stop="Q:", max_tokens=20)

    return lm

In [22]:
llama2 + simil(sentences=["Grass is green", "Football players play on grass pitch"])